In [ ]:
# Connect to MySQL Database 'final_ds' using MySQL Connector
import mysql.connector
connection= mysql.connector.connect(
 host= "localhost",
 user= "root",
 password="",
 database="final_ds"
 )
print(connection)
mycursor = connection.cursor()

In [ ]:
# Retrieve and display all tables in the 'final_ds' database
mycursor.execute("SHOW TABLES;")

In [ ]:
# Fetch and print all table names from the 'final_ds' database
tables = mycursor.fetchall()

print("Tables in 'final_ds' database:")
for table in tables:
    print(table[0])

In [ ]:
# Import pandas for data manipulation and analysis
import pandas as pd

In [ ]:
# Create 'joined_1' table by joining 'transaction' and 'user' tables
query_1 = """
CREATE TABLE IF NOT EXISTS joined_1 AS
SELECT t.TransactionId, t.UserId, t.VisitYear, t.VisitMonth, t.VisitMode, t.AttractionId, t.Rating,
       u.ContenentId, u.RegionId, u.CountryId, u.CityId
FROM transaction t
INNER JOIN user u ON t.UserId = u.UserId;
"""

mycursor.execute(query_1)
connection.commit()
print("Step 1: Created table `joined_1` (transaction + user)")


In [ ]:
# Create 'joined_2' table by joining 'joined_1' with 'continent' table
query_2 = """
CREATE TABLE IF NOT EXISTS joined_2 AS
SELECT j1.*, c.Contenent
FROM joined_1 j1
INNER JOIN continent c ON j1.ContenentId = c.ContenentId;
"""

mycursor.execute(query_2)
connection.commit()
print("Step 2: Created table `joined_2` (joined_1 + continent)")


In [ ]:
# Create 'joined_3' table by joining 'joined_2' with 'region' table
query_3 = """
CREATE TABLE IF NOT EXISTS joined_3 AS
SELECT j2.*, r.Region
FROM joined_2 j2
INNER JOIN region r ON j2.RegionId = r.RegionId;
"""

mycursor.execute(query_3)
connection.commit()
print("Step 3: Created table `joined_3` (joined_2 + region)")


In [ ]:
# Create 'joined_4' table by joining 'joined_3' with 'country' table
query_4 = """
CREATE TABLE IF NOT EXISTS joined_4 AS
SELECT j3.*, co.Country
FROM joined_3 j3
INNER JOIN country co ON j3.CountryId = co.CountryId;
"""

mycursor.execute(query_4)
connection.commit()
print("Step 4: Created table `joined_4` (joined_3 + country)")


In [ ]:
# Create 'joined_5' table by joining 'joined_4' with 'city' table
query_5 = """
CREATE TABLE IF NOT EXISTS joined_5 AS
SELECT j4.*, ci.CityName
FROM joined_4 j4
INNER JOIN city ci ON j4.CityId = ci.CityId;
"""

mycursor.execute(query_5)
connection.commit()
print("Step 5: Created table `joined_5` (joined_4 + city)")


In [ ]:
# Create 'joined_6' table by joining 'joined_5' with 'item' (attraction) table
query_6 = """
CREATE TABLE IF NOT EXISTS joined_6 AS
SELECT j5.*, it.Attraction, it.AttractionAddress, it.AttractionTypeId
FROM joined_5 j5
INNER JOIN item it ON j5.AttractionId = it.AttractionId;
"""

mycursor.execute(query_6)
connection.commit()
print("Step 6: Created table `joined_6` (joined_5 + item/attraction)")


In [ ]:
# Create 'joined_7' table by joining 'joined_6' with 'type' table
query_7 = """
CREATE TABLE IF NOT EXISTS joined_7 AS
SELECT j6.*, t.AttractionType
FROM joined_6 j6
INNER JOIN type t ON j6.AttractionTypeId = t.AttractionTypeId;
"""

mycursor.execute(query_7)
connection.commit()
print("Step 7: Created table `joined_7` (joined_6 + type)")


In [ ]:
# Create 'final_table' by joining 'joined_7' with 'mode' table (Final Step) 🎉
query_8 = """
CREATE TABLE IF NOT EXISTS final_table AS
SELECT j7.*, m.VisitMode AS VisitModeName
FROM joined_7 j7
INNER JOIN mode m ON j7.VisitMode = m.VisitModeId;
"""

mycursor.execute(query_8)
connection.commit()
print("Step 8: Created table `final_table` (joined_7 + mode) 🎉 FINAL STEP!")


In [ ]:
# Retrieve data from 'final_table' and convert it into a Pandas DataFrame
query_final = "SELECT * FROM final_table;"
mycursor.execute(query_final)
result_final = mycursor.fetchall()
columns_final = [desc[0] for desc in mycursor.description]

# Convert to DataFrame
df_final = pd.DataFrame(result_final, columns=columns_final)



In [ ]:
df_final.head(25) # Display first few rows

In [13]:
df_final.to_csv("Full Tourism Data.csv",index = False)